In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.7 MB/s eta 0:00:00


In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4393141 sha256=8621c83a6a6d79d6aa69573f595857c3ff6f1987e5ba47bc78b5a95e92e292e2
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
import json
from tqdm import tqdm
import pandas as pd
tqdm.pandas()
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding,Bidirectional,Concatenate
from keras.optimizers import *

from tensorflow.keras.utils import to_categorical

import pickle
from datasets import load_metric

from nltk.translate.bleu_score import corpus_bleu
import fasttext
import tqdm as notebook_tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Read the training data from JSON file
train_data = pd.read_json("/content/drive/MyDrive/internship/asm_train.json", lines=True)
test_data=pd.read_json("/content/drive/MyDrive/internship/asm_test.json", lines=True)

In [ ]:
#converts the JSON data into a Pandas DataFrame
def correct_json_format(json_file_path):
    with open(json_file_path, 'r', encoding='utf-8') as file:
        json_data = file.read()

    # Process the JSON string to insert commas between objects
    json_data = json_data.replace('}\n{', '},{')
    json_list = json.loads(f'[{json_data}]')
    df = pd.json_normalize(json_list)

    return df


In [ ]:
df_train = correct_json_format("/content/drive/MyDrive/internship/asm_train.json")
df_test = correct_json_format("/content/drive/MyDrive/internship/asm_test.json")


In [ ]:
df_train


,unique_identifier,native word,english word,source,score
0,asm1,দীপলিনা,deeplina,Wikidata,NaN
1,asm2,বন্দ্যোপাধ্যায়,bandyopadhyay,Wikidata,NaN
2,asm3,কৃষন,krishan,Wikidata,NaN
3,asm4,বিষয়া,bishaya,Wikidata,NaN
4,asm5,পেমা,pema,Wikidata,NaN
...,...,...,...,...,...
178625,asm178626,বাগিচাসমূহো,bagisaxomuhu,IndicCorp,-0.058495
178626,asm178627,কার্যনির্বাহক,karzonirbahok,IndicCorp,-0.222479
178627,asm178628,এইটি,eiti,IndicCorp,-0.161426
178628,asm178629,চিত্রশালাত,sitroxalat,IndicCorp,-0.111472


In [ ]:
df_test

,unique_identifier,native word,english word,source
0,asm1,লক্ষীনগৰস্থিত,lakhyeenogorsthito,AK-Freq
1,asm2,চতুৰ্থ,soturtho,AK-Freq
2,asm3,এইখন,eikhan,AK-Freq
3,asm4,প্ৰতিমূৰ্তিসমূহ,protimurtixomuh,AK-Freq
4,asm5,প্ৰতিযোগিতাতে,protijugitate,AK-Freq
...,...,...,...,...
5501,asm5502,ঝা,jha,AK-NEI
5502,asm5503,এলেকন,elecon,AK-NEI
5503,asm5504,ডেচাবাথুলা,desabathula,AK-NEI
5504,asm5505,কাৰখেলে,karkhele,AK-NEI


In [ ]:
#subset train DataFrame
subset_df_train=df_train.sample(frac=0.3).reset_index(drop=True)
subset_df_train.shape

(53589, 5)

In [ ]:
df_train=subset_df_train
df_train.shape

(53589, 5)

In [ ]:
 #start symbol "\t" at the beginning of the text and an end symbol "\n" at the end of the text(target_text)
def add_start_end(target_text):
  text = "\t" + target_text + "\n"
  return text


In [ ]:
#the DataFrame df_train will have a new column 'target_ass' containing the original text values from the 'native word' column with a start symbol "\t" at the beginning and an end symbol "\n" at the end of each text value.
df_train['target_ass'] = df_train['native word'].progress_apply(add_start_end)
df_train.head()

100%|██████████| 53589/53589 [00:00<00:00, 524995.81it/s]


,unique_identifier,native word,english word,source,score,target_ass
0,asm134587,বাগানাপাৰা,baganapara,IndicCorp,-0.041089,\tবাগানাপাৰা\n
1,asm93075,নুচুলে,nusule,IndicCorp,-0.123643,\tনুচুলে\n
2,asm108867,চন্দ্ৰিকাই,sondrikai,IndicCorp,-0.138856,\tচন্দ্ৰিকাই\n
3,asm173819,ছিঙিলে,singile,IndicCorp,-0.171640,\tছিঙিলে\n
4,asm97196,বলাৰসকলক,bolarxokolok,IndicCorp,-0.016239,\tবলাৰসকলক\n


In [ ]:
# add only the end token
# add_end takes a string target_text as input and adds an end symbol "\n"
def add_end(target_text):
  text = target_text + "\n"
  return text

In [ ]:
 #adds a new column named 'decoder_target' to the DataFrame df_train. The values in this new column are derived from the values in the 'native word' column.
df_train['decoder_target'] = df_train['native word'].progress_apply(add_end)
df_train.head()


100%|██████████| 53589/53589 [00:00<00:00, 568245.97it/s]


,unique_identifier,native word,english word,source,score,target_ass,decoder_target
0,asm134587,বাগানাপাৰা,baganapara,IndicCorp,-0.041089,\tবাগানাপাৰা\n,বাগানাপাৰা\n
1,asm93075,নুচুলে,nusule,IndicCorp,-0.123643,\tনুচুলে\n,নুচুলে\n
2,asm108867,চন্দ্ৰিকাই,sondrikai,IndicCorp,-0.138856,\tচন্দ্ৰিকাই\n,চন্দ্ৰিকাই\n
3,asm173819,ছিঙিলে,singile,IndicCorp,-0.171640,\tছিঙিলে\n,ছিঙিলে\n
4,asm97196,বলাৰসকলক,bolarxokolok,IndicCorp,-0.016239,\tবলাৰসকলক\n,বলাৰসকলক\n


In [ ]:
def tokenize(lang):
# Create tokenizer
    #(filters)means that all characters in the input text will be considered for tokenization,(char_lvl)each character will be treated as a separate token.
    tokenizer = Tokenizer(filters='',char_level=True)
    # Fit texts
     #creates the vocabulary based on the characters present in the corpus.
    tokenizer.fit_on_texts(lang)
    word_index = tokenizer.word_index
    return tokenizer,word_index

In [ ]:
# Tokenize words
roman_words = df_train['english word']
ass_words = df_train['target_ass']

roman_tokenizer, roman_tokens = tokenize(roman_words)
ass_tokenizer, ass_tokens = tokenize(ass_words)


print(roman_tokens)

print("No.of unique input tokens:", len(roman_tokens))


{'o': 1, 'a': 2, 'r': 3, 'i': 4, 't': 5, 'n': 6, 'u': 7, 'h': 8, 'k': 9, 'e': 10, 'b': 11, 'l': 12, 'm': 13, 'd': 14, 's': 15, 'p': 16, 'x': 17, 'g': 18, 'j': 19, 'y': 20, 'w': 21, 'c': 22, 'f': 23, 'v': 24, 'z': 25, 'q': 26}
No.of unique input tokens: 26


In [ ]:
roman_words[15]

'pordakhon'

In [ ]:
print(ass_tokens )
print("No.of unique output tokens:", len(ass_tokens))


{'\t': 1, '\n': 2, 'া': 3, 'ৰ': 4, '্': 5, 'ি': 6, 'ক': 7, 'ে': 8, 'ন': 9, 'ত': 10, 'ল': 11, 'ব': 12, 'ম': 13, 'প': 14, 'স': 15, 'ো': 16, 'ু': 17, 'দ': 18, 'ী': 19, 'হ': 20, 'ট': 21, 'জ': 22, 'গ': 23, 'ই': 24, 'চ': 25, 'য': 26, 'ৱ': 27, 'শ': 28, 'র': 29, 'ভ': 30, 'ণ': 31, 'খ': 32, 'ষ': 33, 'ধ': 34, 'ৈ': 35, 'ছ': 36, 'ং': 37, 'থ': 38, 'অ': 39, 'ড': 40, 'আ': 41, 'ূ': 42, 'ও': 43, 'ফ': 44, 'উ': 45, 'এ': 46, '়': 47, 'ৃ': 48, 'ঁ': 49, 'ঘ': 50, 'ঞ': 51, 'ঠ': 52, 'ঙ': 53, 'ৌ': 54, 'ৎ': 55, 'ঢ': 56, 'ঃ': 57, 'ঝ': 58, 'ঐ': 59, 'ঔ': 60, 'ঈ': 61, 'ঋ': 62, 'ঊ': 63}
No.of unique output tokens: 63


In [ ]:
# vocab size (no. of unique characters for each script)

roman_vocab = len(roman_tokens)+1
ass_vocab = len(ass_tokens)+1
roman_vocab,ass_vocab

(27, 64)

In [ ]:
roman_words[45],ass_words[45]

('intaki', '\tইন্টাকি\n')

In [ ]:
# convert words to int sequence

roman_words_in_ids = roman_tokenizer.texts_to_sequences(roman_words)
ass_words_in_ids = ass_tokenizer.texts_to_sequences(ass_words)

In [ ]:
roman_words_in_ids[45],ass_words_in_ids[45]

([4, 6, 5, 2, 9, 4], [1, 24, 9, 5, 21, 3, 7, 6, 2])

In [ ]:
roman_words_in_ids[5]

[11, 1, 8, 4, 8, 6, 4, 3, 4, 9, 8, 20, 1, 9]

In [ ]:
roman_words[5]

'bohihnirikhyok'

In [ ]:
ass_words_in_ids[5]


[1, 12, 20, 6, 57, 9, 6, 4, 19, 7, 5, 33, 7, 2]

In [ ]:
# y_train
# only with end token

label_words = df_train["decoder_target"]
print(label_words[6])
#convert the text sequences in label_words into sequences of integers
y_train = ass_tokenizer.texts_to_sequences(label_words)
y_train

In [ ]:
y_train[9]

[44, 8, 9, 15, 7, 11, 8, 2]

In [ ]:
print(label_words[9])

ফেনসকলে



In [ ]:
# max sequence length

max_encoder_seq_length_roman = df_train['english word'].str.len().max()
max_decoder_seq_length_ass = df_train['target_ass'].str.len().max()

max_encoder_seq_length_roman, max_decoder_seq_length_ass

(23, 21)

In [ ]:
def post_padding(data_in_int_seq, max_seq_len):
  return pad_sequences(data_in_int_seq, max_seq_len, padding = "post")
# padding

roman_padded = post_padding(roman_words_in_ids, max_encoder_seq_length_roman)
ass_padded = post_padding(ass_words_in_ids, max_decoder_seq_length_ass)
y_train_padded = post_padding(y_train, max_decoder_seq_length_ass)

print(roman_padded[16], len(roman_padded[16]))
print(ass_padded[16], len(ass_padded[16]))
print(y_train_padded[16],len(y_train_padded[16]))

[15 22  4 10  6 22 10 19 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0] 23
[ 1 25  3 24  9  5 15  8 22  8  2  0  0  0  0  0  0  0  0  0  0] 21
[25  3 24  9  5 15  8 22  8  2  0  0  0  0  0  0  0  0  0  0  0] 21


In [ ]:
latent_dim = 64
emb_dim = 100

In [ ]:
# Define the encoder model
encoder_inputs = Input(shape=(max_encoder_seq_length_roman,), name="encoder_input")
emb_encoder = Embedding(roman_vocab, emb_dim, mask_zero=True, name="encoder_embedding")(encoder_inputs)
#encoder_lstm = Bidirectional(LSTM(latent_dim, return_sequences=False,return_state=True, name="encoder_bilstm_1"))(emb_encoder)
encoder_outputs, forward_h, forward_c, backward_h, backward_c = Bidirectional(LSTM(latent_dim, return_state=True, return_sequences=False),name="BiLSTM",merge_mode="mul")(emb_encoder)
#encoder_lstm_final = Bidirectional(LSTM(latent_dim, return_state=True, name="encoder_bilstm_2"))(encoder_lstm)
#encoder_states = [encoder_lstm_final[0], encoder_lstm_final[1]]

In [ ]:
# concatenate along the last dimension
encoder_h = Concatenate(axis=-1,name="concat_h")([forward_h, backward_h])
encoder_c = Concatenate(axis=-1,name="concat_c")([forward_c, backward_c])
encoder_h, encoder_c


(<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'concat_h')>,
 <KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'concat_c')>)

In [ ]:
encoder_states = [encoder_h,encoder_c]
encoder_states

[<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'concat_h')>,
 <KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'concat_c')>]

In [ ]:
# decoder with two lstm layers

decoder_inputs = keras.Input(shape=(None,),name="decoder_input")

decoder_emb = Embedding(ass_vocab, emb_dim, mask_zero=True, name="decoder_embedding")(decoder_inputs)
d_lstm_out,d_h,d_c = LSTM(128, return_state=True, return_sequences=True, name="decoder_lstm_1")(decoder_emb, initial_state=encoder_states)
#d_lstm_out1 = LSTM(latent_dim, return_state=False, return_sequences=True,name="decoder_lstm_2")(d_lstm_out, initial_state=context_second)
decoder_dense = keras.layers.Dense(ass_vocab, activation="softmax")

decoder_outputs = decoder_dense(d_lstm_out)

In [ ]:
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01, rho=0.9)

model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 23)]         0           []                               
                                                                                                  
 encoder_embedding (Embedding)  (None, 23, 100)      2700        ['encoder_input[0][0]']          
                                                                                                  
 decoder_input (InputLayer)     [(None, None)]       0           []                               
                                                                                                  
 BiLSTM (Bidirectional)         [(None, 64),         84480       ['encoder_embedding[0][0]']      
                                 (None, 64),                                                  

In [ ]:
roman_padded[45]

array([4, 6, 5, 2, 9, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0], dtype=int32)

In [ ]:
roman_words[45]

'intaki'

In [ ]:

ass_padded[45]

array([ 1, 24,  9,  5, 21,  3,  7,  6,  2,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0], dtype=int32)

In [ ]:
y_train_padded[53587]

array([14,  5,  4, 27, 51,  5, 25,  7, 21, 16,  2,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0], dtype=int32)

In [ ]:
batch_size = 64
epochs = 100


model.fit(
    [roman_padded, ass_padded],
    y_train_padded,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
model.save("/content/drive/MyDrive/internship/mt_bilstm_1l.h5")

Epoch 1/50
670/670 [==============================] - 123s 183ms/step - loss: 0.1318 - accuracy: 0.9514 - val_loss: 0.2524 - val_accuracy: 0.9241
Epoch 2/50
670/670 [==============================] - 121s 181ms/step - loss: 0.1312 - accuracy: 0.9517 - val_loss: 0.2561 - val_accuracy: 0.9226
Epoch 3/50
670/670 [==============================] - 120s 179ms/step - loss: 0.1312 - accuracy: 0.9516 - val_loss: 0.2574 - val_accuracy: 0.9230
Epoch 4/50
670/670 [==============================] - 122s 182ms/step - loss: 0.1312 - accuracy: 0.9515 - val_loss: 0.2550 - val_accuracy: 0.9238
Epoch 5/50
670/670 [==============================] - 120s 180ms/step - loss: 0.1307 - accuracy: 0.9516 - val_loss: 0.2559 - val_accuracy: 0.9229
Epoch 6/50
670/670 [==============================] - 121s 181ms/step - loss: 0.1308 - accuracy: 0.9518 - val_loss: 0.2566 - val_accuracy: 0.9238
Epoch 7/50
670/670 [==============================] - 122s 182ms/step - loss: 0.1328 - accuracy: 0.9512 - val_loss: 0.2518 -

In [ ]:
#Load saved model
saved_model = tf.keras.models.load_model('/content/drive/MyDrive/internship/mt_bilstm_1l.h5')
saved_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 23)]         0           []                               
                                                                                                  
 encoder_embedding (Embedding)  (None, 23, 100)      2700        ['encoder_input[0][0]']          
                                                                                                  
 decoder_input (InputLayer)     [(None, None)]       0           []                               
                                                                                                  
 BiLSTM (Bidirectional)         [(None, 64),         84480       ['encoder_embedding[0][0]']      
                                 (None, 64),                                                  

In [ ]:
inputs = saved_model.get_layer('encoder_input').output                    #Encoder input
bi_lstm_out,f_h,f_c,b_h,b_c = saved_model.get_layer('BiLSTM').output   #Encoder LSTM output

concate_h = saved_model.get_layer('concat_h').output
concate_c = saved_model.get_layer('concat_c').output


In [ ]:
bi_lstm_out

In [ ]:

targets = saved_model.get_layer('decoder_input').output           #Decoder input
embedding_layer = saved_model.get_layer('decoder_embedding')      #Decoder Embedding layer
decoder_lstm1 = saved_model.get_layer('decoder_lstm_1')           #Decoder LSTM layer
dense1 = saved_model.get_layer('dense_1')

ValueError: ignored

In [ ]:
#Encoder

encoder = keras.Model(inputs, [concate_h, concate_c])
encoder.summary()

In [ ]:
#Decoder

decoder_input_h1 = Input(shape=(128,))
decoder_input_c1 = Input(shape=(128,))

x = embedding_layer(targets)


In [ ]:
x

In [ ]:
x,d_out_h,d_out_c = decoder_lstm1(x, initial_state=[decoder_input_h1, decoder_input_c1])

In [ ]:
#x = decoder_lstm2(x, initial_state=[decoder_input_h1, decoder_input_c1])

x = dense1(x)
x.shape


In [ ]:
decoder = keras.Model([targets]+[decoder_input_h1, decoder_input_c1] , [x] + [d_out_h, d_out_c])

decoder.summary()

In [ ]:
reverse_input_char_index = dict((i, char) for char, i in roman_tokens.items())
reverse_target_char_index = dict((i, char) for char, i in ass_tokens.items())

In [ ]:
reverse_input_char_index

In [ ]:
reverse_target_char_index

In [ ]:
v = np.reshape(roman_padded[4], (1, -1))


In [ ]:
encoder.predict(v,verbose=0)

In [ ]:
def predict_list_of_words(list_source_words_in_padded_int_seq):
  list_pred_words = []
  start_token = np.zeros((1, 1))
  start_token[0] = ass_tokenizer.word_index['\t']
  print(start_token)

  for x in tqdm(range(len(list_source_words_in_padded_int_seq))):
    source_seq = start_token
    pred_word = ''
    v = np.reshape(list_source_words_in_padded_int_seq[x], (1, -1))
    next_h, next_c = encoder.predict(v,verbose=0)
    for i in range(max_decoder_seq_length_ass):
      output, next_h, next_c = decoder.predict([source_seq] + [next_h, next_c],verbose=0)
      next_token = np.argmax(output[0, 0, :])
      next_char = reverse_target_char_index[next_token]
      if next_char == '\n':
        break
      else:
        pred_word += next_char
        source_seq = np.zeros((1, 1))
        source_seq[0] = next_token
    list_pred_words.append(pred_word)
  return list_pred_words

In [ ]:
predicted = predict_list_of_words(roman_padded[:20])

In [ ]:
df_analyse = pd.DataFrame()
df_analyse["source"] = roman_words[:20]
df_analyse["predicted"] = predicted
df_analyse["ground_truth"] = df_train["native word"]

df_analyse

In [ ]:
test_roman_words = df_test['english word']
test_target_words = df_test['native word']
test_roman_words[2],test_target_words[2]

In [ ]:
test_roman_words_in_ids = roman_tokenizer.texts_to_sequences(test_roman_words)
test_target_words_in_ids = ass_tokenizer.texts_to_sequences(test_target_words)

print(test_roman_words_in_ids[4])
print(test_target_words_in_ids[4])

In [ ]:
#2. padding

test_roman_padded = post_padding(test_roman_words_in_ids, max_encoder_seq_length_roman)
test_target_padded = post_padding(test_target_words_in_ids, max_decoder_seq_length_ass)

print(test_roman_padded[2])
print(test_target_padded[2])

In [ ]:
len(test_roman_padded[2]), len(test_target_padded[2])

In [ ]:
# prediction on test set

predicted_test = predict_list_of_words(test_roman_padded[:1500])

In [ ]:
# analyse

df_analyse_test = pd.DataFrame()
df_analyse_test["source"] = test_roman_words[:1500]
df_analyse_test["predicted"] = predicted_test
df_analyse_test["ground_truth_label"] = df_test["native word"][:1500]

df_analyse_test.head(10)

In [ ]:
# Calculate accuracy

accuracy = (df_analyse_test['predicted'] == df_analyse_test['ground_truth_label']).mean()

print(f'Test Accuracy = {accuracy*100:.2f}%')

In [ ]:

def split_characters(text_string):
  return [character for character in text_string]

split_characters('চ' 'তু' 'ৰ্থ')

In [ ]:
df_analyse_test["predicted_tokenized"] = df_analyse_test["predicted"].apply(split_characters)
df_analyse_test["label_tokenized"] = df_analyse_test["ground_truth_label"].apply(split_characters)

df_analyse_test.head()

In [ ]:
def create_reference(tokenized):
  return [tokenized]

df_analyse_test["reference"] = df_analyse_test["label_tokenized"].apply(create_reference)
df_analyse_test.head()

In [ ]:
# BLEU Score

# Convert DataFrame columns to lists
predictions = df_analyse_test['predicted_tokenized'].tolist()
true_labels = df_analyse_test['reference'].tolist()


In [ ]:
true_labels

In [ ]:
predictions

In [ ]:
bleu = load_metric("bleu")
bleu.compute(predictions = predictions, references = true_labels)


In [ ]:
def transliterate(source_word_in_padded_int_seq):
  #list_pred_words = []
  start_token = np.zeros((1, 1))
  start_token[0] = ass_tokenizer.word_index['\t']
  #print(start_token)

  #for x in tqdm(range(len(list_source_words_in_padded_int_seq))):
  source_seq = start_token
  pred_word = ''
  #v = np.reshape(source_word_in_padded_int_seq, (1, -1))
  next_h, next_c = encoder.predict(source_word_in_padded_int_seq,verbose=0)
  for i in range(max_decoder_seq_length_ass):
    output, next_h, next_c = decoder.predict([source_seq] + [next_h, next_c],verbose=0)
    next_token = np.argmax(output[0, 0, :])
    next_char = reverse_target_char_index[next_token]
    if next_char == '\n':
      break
    else:
      pred_word += next_char
      source_seq = np.zeros((1, 1))
      source_seq[0] = next_token
  #list_pred_words.append(pred_word)
  return pred_word

input_word = input("\nEnter a romanized word: ")
input_word_in_ids = roman_tokenizer.texts_to_sequences([input_word])

input_word_in_ids_padded = post_padding(input_word_in_ids, max_encoder_seq_length_roman)

print(f"\nTransliterated word : {transliterate(input_word_in_ids_padded)}")



Enter a romanized word: shahir


NameError: ignored